In [1]:
def iou_xyxy(a, b, eps=1e-12):
    # a,b: [x1,y1,x2,y2]
    ax1, ay1, ax2, ay2 = a
    bx1, by1, bx2, by2 = b

    # intersection
    inter_x1 = max(ax1, bx1)
    inter_y1 = max(ay1, by1)
    inter_x2 = min(ax2, bx2)
    inter_y2 = min(ay2, by2)
    iw = max(0.0, inter_x2 - inter_x1)
    ih = max(0.0, inter_y2 - inter_y1)
    inter = iw * ih

    # areas
    aw = max(0.0, ax2 - ax1)
    ah = max(0.0, ay2 - ay1)
    bw = max(0.0, bx2 - bx1)
    bh = max(0.0, by2 - by1)
    area_a = aw * ah
    area_b = bw * bh

    union = area_a + area_b - inter
    if union <= 0.0:   # covers degenerate boxes
        return 0.0
    return inter / (union + eps)

In [ ]:
def ap_score(preds, gts, iou_thresh = 0.5):
    #preds = [id, image, score, box]
    #gts = [id, image, box]
    tp = 0
    fp = 0
    tp_plus_fn = len(gts)
    
    preds.sorted(preds, key=lambda d: d['score'], reverse=True) #Score for now    
    points = [] #[Precision is matches/i , Recall currently matched gt/ total gt]
    
    for p in preds:
        #get image and box
        pimageid, pbox = p['imageid'], p['box']
        
        #Calculate iou for all remaining GT and pick the one with highest if it meets threshold
        image_gts = [
            (gt_id, gt["bbox"])
            for gt_id, gt in gts.items()
            if gt["imageid"] == pimageid
        ]

        # sort in-place, highest IoU first
        image_gts.sort(key=lambda item: iou_xyxy(item[1], pbox), reverse=True)
        
        #If we got a match then remove else 
        if iou_xyxy(image_gts[0], pbox) >= iou_thresh:
            del gts[image_gts[0]]
            tp += 1
        else:
            fp += 1
        
        points.append((tp/(tp+fp), tp/tp_plus_fn))
        
        
        #If ground truth are left over

For the report

Mean Average Precision
Mean AP comes from taking the mean of all AP for the different classes which is a popular metric for object detection. AP for a class is important as it measures a high precision and recall at the same time. It is important to have both as the object detection model otherwise can game the metrics. For instance, by only looking at recall, the model can make extremely many wild guesses where it can get all ground truths, while only looking at precision it can only consider those it's extremely confident about, but miss a lot of other ground truths.

The algorithm for this project takes all predictions from all images with their confidence and boxes and sorts predictions based on confidence. This is favorable for AP to get as many true positives as we can in the beginning. When matching the image, IoU is checked to find a Ground truth. Lastly, in this pothole project the only class is pothole, meaning the outcome is AP for pothole across all images.
